# Direct location 

## PyRugged API - tutorial 1


This tutorial explains how to initialize `PyRugged` and use it to geolocate a satellite image. Let’s imagine the sensor is a single line imager with 2000 pixels and 20° field of view. GPS and AOCS auxiliary data are available and provide us with a list of positions, velocities and attitude quaternions recorded during the acquisition. By passing all this information to `PyRugged`, we will be able to precisely locate each point of the image on the Earth. Well, not exactly precise, as this first tutorial does not use a Digital Elevation Model, but considers the Earth as an ellipsoid. The DEM will be added in a second tutorial: _Direct location with a DEM_. The objective here is limited to explain how to initialize everything `PyRugged` needs to know about the sensor and the acquisition.

---

### Module imports

In [3]:
import orekit_jcc
import numpy as np

from org.hipparchus.geometry.euclidean.threed import Rotation, Vector3D
from org.orekit.frames import FramesFactory
from org.orekit.frames import StaticTransform
from org.orekit.time import AbsoluteDate
from org.orekit.time import TimeScalesFactory
from org.orekit.utils import (
    AngularDerivativesFilter,
    CartesianDerivativesFilter,
    IERSConventions,
    PVCoordinates,
    TimeStampedAngularCoordinates,
    TimeStampedPVCoordinates,
)

from pyrugged.configuration.init_orekit import init_orekit
from pyrugged.intersection.algorithm_id import AlgorithmId
from pyrugged.bodies.body_rotating_frame_id import BodyRotatingFrameId
from pyrugged.bodies.ellipsoid_id import EllipsoidId
from pyrugged.model.inertial_frame_id import InertialFrameId
from pyrugged.model.pyrugged_builder import PyRuggedBuilder
from pyrugged.line_sensor.line_sensor import LineSensor
from pyrugged.line_sensor.linear_line_datation import LinearLineDatation
from pyrugged.los.fixed_rotation import FixedRotation
from pyrugged.los.los_builder import LOSBuilder
from pyrugged.los.sinusoidal_rotation import SinusoidalRotation
from pyrugged.location.optical import OpticalLocation, CorrectionsParams
from pyrugged.intersection.intersection_algorithm import create_intersection_algorithm

### Initialization 

initialize JVM or GraalVM subtrat VM and Orekit, internal orekit data is used here. See configuration section of user manual for further information.

In [4]:
init_orekit()

## 1 - Sensor's definition

Let’s start by defining the imager. The sensor model is described by its viewing geometry (i.e. the line of sight of each physical pixel) and by its datation model.

### Line of sight

We need to define the line of sight (LOS) vector coordinates of each sensor pixel in the satellite frame; X axis is parallel to the velocity vector in the absence of steering, Z is pointing towards the Earth and Y is such that X,Y,Z forms a right-handed coordinate system.

The raw viewing direction of pixel i with respect to the instrument is defined by the vector:

In [5]:
raw_dirs = []
for i in range(2000):
    raw_dirs.append(
        Vector3D(0.0, i * float(np.radians(20)) / 2000.0, 1.0)
    )
  

The instrument is oriented 10° off nadir around the X-axis, we need to rotate the viewing direction to obtain the line of sight in the satellite frame

In [6]:
los_builder = LOSBuilder(raw_dirs)
los_builder.add_los_transform(
    FixedRotation(
        "10-degrees-rotation",
        Vector3D.PLUS_I,
        np.radians(10.)
    )
)


# Here we have considered that the viewing directions are constant with time, 
# it is also possible to have time-dependent lines-of-sight by using other transforms. 
# It is also possible to append several transforms between the raw directions and the final lines-of-sight.
line_of_sight = los_builder.build()

### Datation model

The datation model gives the line number in the image as a function of time, and vice-versa the time as a function of the line number. Here we are using a linear datation model (line sampling rate is constant)

We use Orekit for handling time and dates, and PyRugged for defining the datation model:

In [7]:
gps = TimeScalesFactory.getGPS()
abs_date = AbsoluteDate("2009-12-11T16:59:30.0", gps)
line_datation = LinearLineDatation(
    abs_date, 1.0, 20
)

The LinearLineDataion class is instanciated with 3 arguments: the first is the reference date, the second is the line number at the reference date, and the last argument is the line rate (20 lines per second in this example) .

Note that Orekit can handle many different time scales through TimeScalesFactory, so we do not need to worry about conversion.

#### Sinusoidal Perturbation

You can add sinusoidal perturbation on your LOS when you create them, in order to simulate the vibration of the satellite.

The `SinusoidalRotation` object, is very similar to the `PolynomialRotation`. However make sure to create the time frame before, because you need a reference date to use it. The constructor of this object takes for arguments the following:  
-The name of the transform  :  str  
-The axis around which the (sinusoidal) rotation take place  :  Vector3D
-The reference date to set a time frame for the sinusoide function  :  AbsoluteDate  
-The amplitude of the sinusoide in radians  :  float
-The frequency of the sinusoide in Hz  :  float  
-The phase of the sinusoide  :  float

(We will not use this perturbation in the following)

In [8]:
los_builder_pert = LOSBuilder(raw_dirs)
los_builder_pert.add_los_transform(
    SinusoidalRotation(
        "sinusoidal rotation",
        Vector3D.PLUS_J, #rotation axis
        reference_date = abs_date,
        amplitude = 1e-4,
        frequency = 30,
        phase = 0
    )
)
line_of_sight_pert = los_builder_pert.build()


### Line sensor
With the LOS and the datation now defined, we can initialize a line sensor object in PyRugged:

In [9]:
line_sensor = LineSensor(
    "my-sensor",
    line_datation,
    Vector3D.ZERO,
    line_of_sight,
)

The first parameter is the nickname of the sensor. It is necessary because we can define multiple sensors (useful for an imager with several spectral channels or detector arrays). The third argument is the sensor position relative to the centre of the satellite reference frame. Here it is set to 0.

## 2 - Satellite position, velocity and attitude

As input to Rugged, we will need to pass sufficient information to describe the position and orientation of the platform during the acquisition. In our example, the list of positions, velocities and quaternions are hard-coded. In real life, we would extract GPS and AOCS data from the satellite auxiliary telemetry.

Note that for simulation purpose, we could also use Orekit to simulate the orbit. It offers very convenient functions to propagate sun-synchronous orbits with yaw steering compensation (typical orbits for Earth Observation satellites).

### Reference frames

Rugged expects the positions (unit: m), velocities (unit: m/s) and quaternions to be expressed in an inertial frame. All standard inertial and terrestrial frames are implemented in Orekit, so we just need to specify which frames we are using and convert if necessary.

Conversion from inertial to Earth-rotating frame is transparent to the user and is based on the most recent precession/nutation model on top of which corrections published by the IERS are applied. IERS bulletins and other physical data are provided within the orekit data folder. There are several ways to configure Orekit to use this data.

In our application, we simply need to know the name of the frames we are working with. Positions and
velocities are given in the ITRF terrestrial frame, while the quaternions are given in EME2000
inertial frame.

In [10]:
eme_2000 = FramesFactory.getEME2000()

# We don't want to compute tiny tidal effects at millimeter level
simple_eop = True

itrf = FramesFactory.getITRF(
    IERSConventions.IERS_2010, simple_eop
)

### Satellite attitude

The attitude quaternions are grouped in a list of TimeStampedAngularCoordinates objects:

Each attitude sample (quaternion, time) is added to the list.

In [11]:
def add_satellite_q(gps, satellite_q_list, abs_date, q_0, q_1, q_2, q_3):
    """Add element to satellite_q_list argument"""

    attitude_date = AbsoluteDate(abs_date, gps)
    rotation = Rotation(q_0, q_1, q_2, q_3, True)
    pair = TimeStampedAngularCoordinates(attitude_date, rotation, Vector3D.ZERO, Vector3D.ZERO)
    satellite_q_list.append(pair)
    

satellite_q_list = []

add_satellite_q(gps, satellite_q_list, "2009-12-11T16:58:42.592937", -0.340236, 0.333952, -0.844012, -0.245684)
add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:06.592937", -0.354773, 0.329336, -0.837871, -0.252281)
add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:30.592937", -0.369237, 0.324612, -0.831445, -0.258824)
add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:54.592937", -0.3836, 0.319792, -0.824743, -0.265299)
add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:18.592937", -0.397834, 0.314883, -0.817777, -0.271695)
add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:42.592937", -0.411912, 0.309895, -0.810561, -0.278001)
add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:06.592937", -0.42581, 0.304838, -0.803111, -0.284206)
add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:30.592937", -0.439505, 0.299722, -0.795442, -0.290301)
add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:54.592937", -0.452976, 0.294556, -0.787571, -0.296279)
add_satellite_q(gps, satellite_q_list, "2009-12-11T17:02:18.592937", -0.466207, 0.28935, -0.779516, -0.302131)

### Positions and velocities

Similarly the positions and velocities will be set in a list of TimeStampedPVCoordinates.

In [12]:
def add_satellite_pv(gps, eme2000, itrf, satellite_pv_list, abs_date, p_x, p_y, p_z, v_x, v_y, v_z):
    """Add element to satellite_pv_list"""

    ephemeris_date = AbsoluteDate(abs_date, gps)
    position = Vector3D(p_x, p_y, p_z)
    velocity = Vector3D(v_x, v_y, v_z)
    pv_itrf = PVCoordinates(position, velocity)
    transform = StaticTransform.cast_(itrf.getTransformTo(eme2000, ephemeris_date))
    p_eme_2000 = transform.transformPosition(pv_itrf.getPosition())
    v_eme_2000 = transform.transformVector(pv_itrf.getVelocity())
    satellite_pv_list.append(TimeStampedPVCoordinates(ephemeris_date, p_eme_2000, v_eme_2000, Vector3D.ZERO))


satellite_pv_list = []

add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:58:42.592937", -726361.466, -5411878.485, 4637549.599, -2463.635, -4447.634, -5576.736)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:04.192937", -779538.267, -5506500.533, 4515934.894, -2459.848, -4312.676, -5683.906)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:25.792937", -832615.368, -5598184.195, 4392036.13, -2454.395, -4175.564, -5788.201)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:47.392937", -885556.748, -5686883.696, 4265915.971, -2447.273, -4036.368, -5889.568)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:08.992937", -938326.32, -5772554.875, 4137638.207, -2438.478, -3895.166, -5987.957)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:30.592937", -990887.942, -5855155.21, 4007267.717, -2428.011, -3752.034, -6083.317)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:52.192937", -1043205.448, -5934643.836, 3874870.441, -2415.868, -3607.05, -6175.6)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:13.792937", -1095242.669, -6010981.571, 3740513.34, -2402.051, -3460.291, -6264.76)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:35.392937", -1146963.457, -6084130.93, 3604264.372, -2386.561, -3311.835, -6350.751)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:56.992937", -1198331.706, -6154056.146, 3466192.446, -2369.401, -3161.764, -6433.531)
add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:02:18.592937", -1249311.381, -6220723.191, 3326367.397, -2350.574, -3010.159, -6513.056)

## 3 - PyRugged initialization

`PyRugged` object contains platform (frame transform between spacecraft,inertial and body frame), instrument and body elements.
As the Rugged top level object that will be used for all user interaction is quite involved and can be built in several different ways, a builder pattern approach has been adopted. The builder itself is configured by calling dedicated setters for each concept (Digital Elevation Model, intersection algorithm, trajectory, …). As all these concepts can be chained together, the setters themselves implement the fluent interface pattern, which implies each setter returns the builder instance, and therefore another setter can be called directly.

__Setters__


- The `set_ellipsoid` setter defines the shape and orientation of the ellipsoid. We use simple predefined enumerates: `EllipsoidId.WGS84`, `InertialFrameId.EME2000`, but could also use a custom ellipsoid if needed.

- The `set_time_span` setter is used to define the global time span of the search algorithms (direct and inverse location). Four parameters are used for this: `acquisition_start_date`, `acquisition_stop_date`, `t_step` (step at which the pre-computed frames transforms cache will be filled), `time_tolerance` (margin allowed for extrapolation during inverse location, in seconds. The `t_step` parameter is a key parameter to achieve good performances as frames transforms will be precomputed throughout the time span using this time step. These computation are costly because they involve Earth precession/nutation models to be evaluated. So the transformed are precomputed and cached instead of being recomputed for each pixel. However, if direct and inverse location are expected to be performed over a reduced time span (say a few tens of seconds), precomputing the transforms over half an orbit at one millisecond rate would be a waste of computing power. Typical values are therefore to restrict the time span as much as possible to properly cover the expected direct and inverse location calls, and to use a step between one millisecond and one second, depending on the required accuracy. The exact value to use is mission-dependent. The final `time_tolerance` parameter is simply a margin used before and after the final precomputed transforms to allow a slight extrapolation if during a search the interval is slightly overshoot. A typical value is to allow a few images lines so for example a 5 lines tolerance would imply computing the tolerance as: `time_tolerance = 5 / line_sensor.get_rate(0))`.

- The `set_trajectory` setter defines the spacecraft evolution. The arguments are the list of time-stamped positions and velocities as well as the inertial frame with respect to which they are defined and options for interpolation: number of points to use and type of filter for derivatives. The interpolation polynomials for `nb_pv_points` without any derivatives (case of `CartesianDerivativesFilter.USE_P`: only positions are used, without velocities) have a degree `nb_pv_points - 1`. In case of computation with velocities included (case of `CartesianDerivativesFilter.USE_PV`), the interpolation polynomials have a degree `2 * nb_pv_points - 1`. If the positions/velocities data are of good quality and separated by a few seconds, one may choose only a few points but interpolate with both positions and velocities; in other cases, one may choose more points but interpolate only with positions. We find similar arguments for the attitude quaternions.

__Line sensors__

The last setter used, `add_line_sensor`, registers a line sensor. As can be deduced from its prefix (add instead of set), it can be called several time to register several sensors that will all be available in the built Rugged instance. We have called the method only once here, so we will use only one sensor.

__Building PyRugged instance__

After the last setter has been called, we call the `build()` method which really build the `PyRugged` instance (and not a `PyRuggedBuilder` instance has the setter did).

The various setters can be called in any order. The only important thing is that once a `PyRugged` instance has been created by calling the `build()` method, it is independent from the builder so later calls to setters will not change the build instance. In fact, it is possible to create a builder, then call its `build()` method to create a first `PyRugged` instance, and then to modify the builder configuration by calling again some of the setters and building a second `PyRugged` instance from the new configuration. This allows to perform comparisons between two different configurations in the same program and without having to recreate everything.

In [13]:
builder = PyRuggedBuilder()


builder.set_ellipsoid(
    new_ellipsoid = None,
    ellipsoid_id = EllipsoidId.WGS84,
    body_rotating_frame_id = BodyRotatingFrameId.ITRF,
)

builder.set_time_span(
    abs_date,
    abs_date.shiftedBy(60.0),
    0.01,
    5 / line_sensor.get_rate(0)
)

builder.set_trajectory(
    satellite_pv_list,
    4, CartesianDerivativesFilter.USE_P,
    satellite_q_list, 
    4, AngularDerivativesFilter.USE_R,
    inertial_frame_id=InertialFrameId.EME2000,
)

builder.add_sensor(line_sensor)

rugged = builder.build()

## 4 - optical path and intersection algorithm

`OpticalLocation` object used `PyRugged` object to perform location function with configured intersection algorithm taking into account corrections

__Intersection algorithm__

- The `create_intersection_algorithm` helps to specifies the intersection algorithm to use. As this tutorial is intended to be very simple for a beginning, we choose to use directly the ellipsoid and not a real Digital Elevation Model, so we can use `AlgorithmId.IGNORE_DEM_USE_ELLIPSOID` as the single parameter of this setter.

- The `tile_updater` element specifies the Digital Elevation Model. In fact, as we decided to ignore the Digital Elevation Model in this tutorial, we could have omitted this call and it would have worked correctly. We preferred to let it in so users do not forget to set the Digital Elevation Model for intersection algorithms that really use them. As the model will be ignored, we can put the parameters for this setter to `None` and `0`. Of course if another algorithm had been chosen, `None` parameters would clearly not work, this is explained in another tutorial: _Direct location with a DEM_.

__Default corrections__

Rugged takes into account by default some corrections for more accurate locations:

- `light_time_correction` (compensates or not light time between ground and spacecraft).
- `aberration_of_light_correction` (compensates or not aberration of light, which is velocity composition between light and spacecraft when the light from ground points reaches the sensor).
- `atmospheric refraction` (compensates the atmospheric path see dedicated notebook)

Not compensating the delay or the velocity composition are mainly useful for validation purposes against system that do not compensate it. When the pixels line of sight already includes the aberration of light correction, one must obviously deactivate the correction.

a corrections parameters tuple `CorrectionsParams` is used to specifes which effect has to be activated or not.

In [14]:
correction_params = CorrectionsParams(False, False, None)
intersection_algorithm = create_intersection_algorithm(AlgorithmId.IGNORE_DEM_USE_ELLIPSOID)

location = OpticalLocation(rugged, intersection_algorithm, correction_params)

## 5 - Direct location of a single sensor pixel

Finally everything is set to do some real work. Let’s try to locate a point on Earth for upper left point (first line, first pixel):

In [15]:
#sensor name is optional since PyRugged object conains only one snesor
longitudes,latitudes, altitudes = location.direct_location([0], [0],"my-sensor")


print("longitude {:.8f}° latitude {:.8f}° altitude {:.2f}m".format(np.degrees(longitudes[0]), np.degrees(latitudes[0]),altitudes[0]))

longitude -96.94914729° latitude 37.58499530° altitude 0.00m


one can use directly line of sight information to compute direct location

In [16]:
position = line_sensor.position
first_line_date = line_sensor.get_date(0)
los = line_sensor.get_los(first_line_date, 0)

up_left_point = location.direct_location_of_los(
    first_line_date,
    position, 
    los,
)

print("longitude {:.8f}° latitude {:.8f}° altitude {:.2f}m".format(np.degrees(up_left_point[0]), np.degrees(up_left_point[1]),up_left_point[2]))

longitude -96.94914729° latitude 37.58499530° altitude 0.00m
